In [ ]:
from google.colab import drive
drive.mount('/content/google_drive')

Mounted at /content/google_drive


In [ ]:
!nvidia-smi

Tue Dec 13 04:57:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -U "tensorflow>=2.5"


<div><div class="highlight-default notranslate"><div class="highlight"><pre><span></span>content/
├─ scripts/ (Google Drive Mount)
│  └─ preprocessing/
├─ models/
│  ├─ community/
│  ├─ official/
│  ├─ orbit/
│  ├─ research/
│  └─ ...
└─ workspace/ (Google Drive Mount)
    └─ training_demo/
</pre></div>


<div><div class="highlight-default notranslate"><div class="highlight"><pre><span></span>training_demo/
├─ annotations/
├─ exported-models/
├─ images/
│  ├─ test/
│  └─ train/
├─ models/
├─ pre-trained-models/
└─ README.md
</pre></div>

In [ ]:
import tensorflow as tf

In [ ]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen


# import tensorflow_hub as hub

tf.get_logger().setLevel('ERROR')

In [ ]:
import re
from shutil import copyfile
import argparse
import math
import random

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

In [ ]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3630, done.
remote: Counting objects: 100% (3630/3630), done.
remote: Compressing objects: 100% (3028/3028), done.
remote: Total 3630 (delta 958), reused 1551 (delta 547), pack-reused 0
Receiving objects: 100% (3630/3630), 47.12 MiB | 8.54 MiB/s, done.
Resolving deltas: 100% (958/958), done.


In [ ]:
%%bash
sudo apt install -y protobuf-compiler
cd /content/models/research/
protoc object_detection/protos/*.proto --python_out=.
cd /content
git clone https://github.com/cocodataset/cocoapi.git
cd /content/cocoapi/PythonAPI
make
cp -r pycocotools /content/models/research/
cd /content/models/research/
cp object_detection/packages/tf2/setup.py .
python -m pip install .
cd /content

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
output_directory = '/content/tf_lite'

In [ ]:
!python /content/models/research/object_detection/export_tflite_graph_tf2.py \
--trained_checkpoint_dir="/content/google_drive/MyDrive/RetinaNet_prac/workspace/exported_models/checkpoint" \
--output_directory='/content/tf_lite' \
--pipeline_config_path="/content/google_drive/MyDrive/RetinaNet_prac/workspace/exported_models/pipeline.config"

In [ ]:
# !python /content/models/research/object_detection/export_tflite_graph_tf2.py \
# --trained_checkpoint_dir="/content/google_drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/exported-models/saved_efficientDet_D1/checkpoint" \
# --output_directory='/content/tf_lite2' \
# --pipeline_config_path="/content/google_drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/exported-models/saved_efficientDet_D1/pipeline.config"

In [ ]:
!ls -lah /content/tf_lite/saved_model/

In [ ]:
!tflite_convert --saved_model_dir=/content/tf_lite/saved_model/ --output_file=tf_lite/model.tflite

In [ ]:
!ls -lah tf_lite/model.tflite

-rw-r--r-- 1 root root 195M Dec 16 06:59 tf_lite/model.tflite


In [ ]:
# based on https://github.com/tensorflow/examples/blob/master/lite/examples/object_detection/raspberry_pi/detect_picamera.py

from PIL import Image, ImageDraw, ImageFont
import re, glob
from six import BytesIO
import numpy as np
%matplotlib inline

def draw_image(image, results, size):
    result_size = len(results)
    for idx, obj in enumerate(results):
        # Prepare image for drawing
        draw = ImageDraw.Draw(image)

        # Prepare boundary box
        xmin, ymin, xmax, ymax = obj['bounding_box']
        xmin = int(xmin * size[1])
        xmax = int(xmax * size[1])
        ymin = int(ymin * size[0])
        ymax = int(ymax * size[0])

        # Draw rectangle to desired thickness
        for x in range( 0, 4 ):
            draw.rectangle((ymin, xmin, ymax, xmax), outline=(255, 255, 0), width=5)

    displayImage = np.asarray( image )
    display(Image.fromarray(displayImage))

def load_labels(path):
    """Loads the labels file. Supports files with or without index numbers."""
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        labels = {}
        for row_number, content in enumerate(lines):
            pair = re.split(r'[:\s]+', content.strip(), maxsplit=1)
            if len(pair) == 2 and pair[0].strip().isdigit():
                labels[int(pair[0])] = pair[1].strip()
            else:
                labels[row_number] = pair[0].strip()
    return labels

def set_input_tensor(interpreter, image):
    """Sets the input tensor."""
    tensor_index = interpreter.get_input_details()[0]['index']
    input_tensor = interpreter.tensor(tensor_index)()[0]
    input_tensor[:, :] = image


def get_output_tensor(interpreter, index):
    """Returns the output tensor at the given index."""
    output_details = interpreter.get_output_details()[index]
    tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
    return tensor


def detect_objects(interpreter, image, threshold):
    """Returns a list of detection results, each a dictionary of object info."""
    set_input_tensor(interpreter, image)
    interpreter.invoke()

    # Get all output details
    scores = get_output_tensor(interpreter, 0)   #0:scores
    boxes = get_output_tensor(interpreter, 1) #1:boxes
    count = int(get_output_tensor(interpreter, 2)) #2:count
    classes = get_output_tensor(interpreter, 3) #3:classes
    # count = int(get_output_tensor(interpreter, 3))
    print("boxes=", boxes)
    print("classes=", classes)
    print("scores=", scores)
    print("count=", count)

    results = []
    for i in range(count):
        if scores[i] >= threshold:
            result = {
                'bounding_box': boxes[i],
                'class_id': classes[i],
                'score': scores[i]
            }
            results.append(result)
    
    return results

In [ ]:
interpreter = tf.lite.Interpreter(model_path="tf_lite/model.tflite")
interpreter.allocate_tensors()
_, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

In [ ]:
!pwd

/content


In [ ]:
!ls -al /content/testimages/

In [ ]:
input_mean = 0
input_std = 1

for i, image_path in enumerate(glob.glob('/content/testimages/*.jpg')):
  print(image_path)
  image = Image.open(image_path)
  image_pred = image.resize((input_width ,input_height), Image.ANTIALIAS)
  if interpreter.get_input_details()[0]['dtype'] == np.float32:
    image_pred = (np.float32(image_pred) - input_mean) / input_std
  results = detect_objects(interpreter, image_pred, 0.2)

  draw_image(image, results, image.size)

In [ ]:
from object_detection.utils import label_map_util

In [ ]:
_ODT_LABEL_MAP_PATH = '/content/google_drive/MyDrive/RetinaNet_prac/workspace/annotations/label_map.pbtxt'
_TFLITE_LABEL_PATH = "/content/tflite_label_map.txt"

category_index = label_map_util.create_category_index_from_labelmap(
    _ODT_LABEL_MAP_PATH)
f = open(_TFLITE_LABEL_PATH, 'w')
for class_id in range(1, 38):   ###############
  if class_id not in category_index:
    f.write('???\n')
    continue
  name = category_index[class_id]['name']
  f.write(name+'\n')
f.close()

In [ ]:
!pip install tf-nightly -q

     |████████████████████████████████| 556.5 MB 3.0 kB/s 
     |████████████████████████████████| 1.7 MB 80.3 MB/s 
     |████████████████████████████████| 439 kB 76.4 MB/s 
     |████████████████████████████████| 5.7 MB 80.7 MB/s 


In [ ]:
# !pip install tflite-support

In [ ]:
!pip install tflite_support_nightly

In [ ]:
!pip install -U flatbuffers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import flatbuffers

In [ ]:
from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils

ObjectDetectorWriter = object_detector.MetadataWriter

In [ ]:
_MODEL_PATH="/content/model.tflite"

In [ ]:
_TFLITE_MODEL_WITH_METADATA_PATH = '/content/model_with_metadata.tflite'

In [ ]:
_TFLITE_LABEL_PATH="/content/tflite_label_map.txt"

In [ ]:
writer = ObjectDetectorWriter.create_for_inference(
    writer_utils.load_file(_MODEL_PATH), [127.5], [127.5],[_TFLITE_LABEL_PATH])

In [ ]:
writer_utils.save_file(writer.populate(), _TFLITE_MODEL_WITH_METADATA_PATH)